# Human Activity Detection (HAR) Dataset: Cursory Comparison of Models

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
from pathlib import Path

import pandas as pd
import umap.plot
import xgboost as xgb
from umap import UMAP

In [ ]:
sys.path.append(
    os.path.join(
        Path(os.getcwd()).parents[0].as_posix(),
        "src"
    )
)

In [ ]:
from data_loader import HARDataLoader

In [ ]:
idx = pd.IndexSlice

# Loading Data

In [ ]:
dl = HARDataLoader()

In [ ]:
dl.load()

# Intro

In [ ]:
dl.df_train_y["name"].value_counts()

In [ ]:
dl.df_train_x.loc[0].reset_index(drop='subject_id').plot(
    y=['body_acc_x', 'body_acc_y', 'body_acc_z']
)

# Baseline Model

- HAR comes with a set of features created from sensor data time series:

In [ ]:
dl.df_train_x_tr.head()

- Features provide a good amount of data separation:

In [ ]:
mapper = UMAP().fit(
    dl.df_train_x_tr.values
)

In [ ]:
umap.plot.points(
    mapper, 
    labels=dl.df_train_y["name"],
    theme='fire'
)

- Baseline classification results using ordinary XGBoost classifier with minimal tuning: